In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install turicreate

     |████████████████████████████████| 91.9MB 68kB/s 
     |████████████████████████████████| 86.3MB 49kB/s 
     |████████████████████████████████| 3.4MB 46.6MB/s 
     |████████████████████████████████| 327kB 48.6MB/s 
     |████████████████████████████████| 3.8MB 40.3MB/s 
     |████████████████████████████████| 450kB 32.5MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320850 sha256=77ec414c6efe4588ac0e483d1a901ee03f82475445cb23c5127e72c4121279ee
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=0cccf6a226cf6739dff87a73ae658a2b43c36656104df8b55a584a13bb5838f8
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built resampy gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Fo

In [ ]:
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten, Concatenate, Dense, Activation,Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam
from scipy.sparse import csr_matrix
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import gzip
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import re
import random
import string
import nltk
import numpy as np
from numpy import *
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
import turicreate as tc
random.seed(10)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Video_Games.json.gz
data = []
with gzip.open('meta_Video_Games.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
df = pd.DataFrame.from_dict(data)
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz
data2 = []
with gzip.open('Video_Games_5.json.gz') as f:
    for l in f:
        data2.append(json.loads(l.strip()))


--2020-06-19 05:00:29--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Video_Games.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80222877 (77M) [application/octet-stream]
Saving to: ‘meta_Video_Games.json.gz’

meta_Video_Games.js 100%[===================>]  76.51M  35.2MB/s    in 2.2s    

2020-06-19 05:00:31 (35.2 MB/s) - ‘meta_Video_Games.json.gz’ saved [80222877/80222877]

--2020-06-19 05:00:39--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154050105 (147M) [application/octet-stream]
Saving to: ‘Video_Games_5.json.gz’

Video_Games_5.json. 100%[===================>] 146.91M  42

# Data Preprocessing

In [ ]:
df2 = pd.DataFrame.from_dict(data2)
df2 = df2[["reviewerID","asin","reviewText", "summary" ,"overall"]]
df2.drop_duplicates(keep=False , inplace = True)
df3 = pd.merge(df, df2, on="asin", how='right')
df4 = df3.groupby("asin").agg({"overall":"mean", "reviewerID":"count"})
df4["avg_rating"]= df4.overall
df4["Count"]= df4.reviewerID
df5 = pd.merge(df3, df4, on="asin", how='inner')
#df5 = df5[df5.Count >10]
df5 = df5[["category","title","asin","reviewerID_x","overall_x","reviewText","summary","avg_rating","Count","brand","feature","description"]]
df7 = df5.groupby("reviewerID_x").agg({"asin":"count"})
df7["user_count"] = df7.asin
df6 = pd.merge(df5, df7, on="reviewerID_x", how='inner')
del(df2,df3,df4,df5, data, data2)
#df_final = df6[df6.user_count >10]
df_final =df6
asin_count =dict(zip(df6.asin_x,df6.Count))
df_final2  =df_final.drop(["reviewText","summary","feature","description","asin_y","category", "brand","user_count","Count","avg_rating"], axis = 1)
df_user_item = df_final2.drop_duplicates( )
df_user_item2 = df_user_item.drop_duplicates(subset=["asin_x","reviewerID_x"])

In [ ]:
df_user_item2["overall"] = df_user_item2["overall_x"].apply(lambda x : x+1 if x ==1 else x)
df_user_item2["likes"] = df_user_item2["overall_x"].apply(lambda x : 0 if x<3.5 else 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:

id_reviewer={}
c=0
for i in df_user_item2.reviewerID_x.unique():
  id_reviewer[i]=c
  c=c+1
id_asin={}
c=0
for i in df_user_item2.asin_x.unique():
  id_asin[i]=c
  c=c+1

In [ ]:
df_user_item2["reviewer_Id"] = df_user_item2["reviewerID_x"].apply(lambda x: id_reviewer[x])
df_user_item2["asin_Id"] = df_user_item2["asin_x"].apply(lambda x: id_asin[x])
id_to_asin=dict(zip(df_user_item2["asin_Id"],df_user_item2["asin_x"]))
id_to_title = dict(zip(df_user_item2["asin_Id"],df_user_item2["title"]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_user_item2.overall.value_counts()

5.0    273978
4.0     83253
2.0     48896
3.0     43928
Name: overall, dtype: int64

In [ ]:

ratings = tc.SFrame(data = df_user_item2)

In [ ]:
ratings.head()

title,asin_x,reviewerID_x,overall_x,overall,likes,reviewer_Id,asin_Id
Anno 2070,0700026657,A1HP7NVNPFMA4N,5.0,5.0,1,0,0
Hexen,B00000JMWB,A1HP7NVNPFMA4N,5.0,5.0,1,0,1
Logitech Dual Action GamePad USB ...,B0000ALFCI,A1HP7NVNPFMA4N,2.0,2.0,0,0,2
Wii,B0009VXBAQ,A1HP7NVNPFMA4N,5.0,5.0,1,0,3
Mario & Luigi: PartnersIn Time ...,B000B8J7K0,A1HP7NVNPFMA4N,3.0,3.0,0,0,4
Super Mario Galaxy,B000FQ9QVI,A1HP7NVNPFMA4N,5.0,5.0,1,0,5
Age Of Mythology: Titans- PC ...,B000P5BSUQ,A1HP7NVNPFMA4N,5.0,5.0,1,0,6
var aPageStart = (newDate()).getTime();\nvar ...,B0018QOWEU,A1HP7NVNPFMA4N,5.0,5.0,1,0,7
var aPageStart = (newDate()).getTime();\nvar ...,B0019R3O7E,A1HP7NVNPFMA4N,5.0,5.0,1,0,8
MySims Kingdom - NintendoWii ...,B0019R4VCQ,A1HP7NVNPFMA4N,5.0,5.0,1,0,9


# Modelling

In [ ]:
training_data, validation_data = tc.recommender.util.random_split_by_user(ratings, 'reviewer_Id', 'asin_Id',item_test_proportion=0.1,max_num_users=2000, )

In [ ]:
model = tc.recommender.ranking_factorization_recommender.create(training_data, user_id='reviewer_Id',item_id='asin_Id',target='likes', binary_target=True )


Preparing data set.

Data has 448427 observations with 55175 users and 17340 items.

Data prepared in: 2.24137s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training Predictive Error.

Final objective value: 1.69723

Final training Predictive Error: 0.532748

In [ ]:
model.evaluate(validation_data)


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0010373443983402492 | 0.0003457814661134163 |
|   2    | 0.0015560165975103733 | 0.0017289073305670815 |
|   3    | 0.0013831258644536651 | 0.0027662517289073307 |
|   4    | 0.0015560165975103731 | 0.0038035961272475808 |
|   5    | 0.0016597510373443976 |  0.004989132582493579 |
|   6    | 0.0017289073305670822 | 0.0054831061055127415 |
|   7    | 0.0017783046828689978 |  0.007039122703023119 |
|   8    |  0.001685684647302905 |  0.007557794902193244 |
|   9    |  0.001498386353158138 |  0.007557794902193243 |
|   10   | 0.0015560165975103738 |  0.008191727590067836 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.40882339813172425

Per User RMSE (best)
+-------------+--------------------+-------

{'precision_recall_by_user': Columns:
 	reviewer_Id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 17352
 
 Data:
 +-------------+--------+-----------+--------+-------+
 | reviewer_Id | cutoff | precision | recall | count |
 +-------------+--------+-----------+--------+-------+
 |      41     |   1    |    0.0    |  0.0   |   5   |
 |      41     |   2    |    0.0    |  0.0   |   5   |
 |      41     |   3    |    0.0    |  0.0   |   5   |
 |      41     |   4    |    0.0    |  0.0   |   5   |
 |      41     |   5    |    0.0    |  0.0   |   5   |
 |      41     |   6    |    0.0    |  0.0   |   5   |
 |      41     |   7    |    0.0    |  0.0   |   5   |
 |      41     |   8    |    0.0    |  0.0   |   5   |
 |      41     |   9    |    0.0    |  0.0   |   5   |
 |      41     |   10   |    0.0    |  0.0   |   5   |
 +-------------+--------+-----------+--------+-------+
 [17352 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_

In [ ]:
valid = validation_data.to_dataframe()
train = training_data.to_dataframe()
valid2 = valid[valid.likes ==1]
df_grp = valid2.groupby("reviewer_Id")

In [ ]:
user_list = valid2.reviewer_Id.unique()
#df_asin = pd.read_csv("drive/My Drive/Data (1)/asin_int_values.csv")
int_to_asin=dict({count:i for i,count in enumerate(df_asin.columns)})


NameError: ignored

In [ ]:
results = model.recommend(users =user_list,k=20,diversity =1, random_seed = 10)
df_recs = results.sort(['reviewer_Id','rank'], ascending=True).to_dataframe()
rec = df_recs.groupby("reviewer_Id")

In [ ]:
df_user_item2[df_user_item2.reviewer_Id==536]

,title,asin_x,reviewerID_x,overall_x,overall,likes,reviewer_Id,asin_Id
12201,X-Men vs. Street Fighter,B000006OVK,A14XH33SGMTA7R,5.0,5.0,1,536,3766
12202,Marvel Super Heroes Vs. Street Fighter,B00000IPEE,A14XH33SGMTA7R,5.0,5.0,1,536,3769
12204,Marvel Vs. Capcom: Clash of Super Heroes,B00004R9IC,A14XH33SGMTA7R,5.0,5.0,1,536,3789
12206,Mega Man X Collection - PlayStation 2,B0009XBWY6,A14XH33SGMTA7R,5.0,5.0,1,536,756
12207,Age Of Empires Collector's Edition (Limited Ed...,B000J18SR2,A14XH33SGMTA7R,5.0,5.0,1,536,44
12208,PlayStation 2 Console Slim PS2,B000TLU67W,A14XH33SGMTA7R,5.0,5.0,1,536,718
12209,AmazonBasics Carrying Case for Nintendo - New ...,B0050SVNZ8,A14XH33SGMTA7R,3.0,3.0,0,536,1065
12210,Saint Seiya Sanctuary Battle (Playstation 3),B007ORIXIS,A14XH33SGMTA7R,5.0,5.0,1,536,5010
12211,Age of Empires III Complete Collection [Download],B00AR05D7I,A14XH33SGMTA7R,5.0,5.0,1,536,5011


In [ ]:
hit =0
c=0
arh =0
for u in user_list:
  c+=1
  for item in df_grp.get_group(u).asin_Id:
    if item in rec.get_group(u).asin_Id.values:
      arh+= 1/(list(rec.get_group(u).asin_Id.values).index(item)+1)
      hit= hit+1
print(hit/c)
print(arh/len(user_list))

536
2596
4651
12465
15711
16642
22927
24622
28219
29789
34280
37222
37934
38697
38790
39476
41271
41456
43508
44174
44477
44767
49285
51149
53695
0.03033980582524272
0.0042665885316419605


In [ ]:
rankings = pd.read_csv('drive/My Drive/Data (1)/Popularity_Rankings_for_Diversity.csv')

In [ ]:

def Novelty(Rankings,user_list):

  print("Novelty:   Average popularity rank of recommended items. Higher means more novel.")

  n=0
  total=0
  for u in tqdm(user_list):
    #the next 5 lines are to just generate the top n recommendations
    recs = rec.get_group(u).asin_Id.values
    recommendations=[id_to_asin[i] for i in recs]
    for item in recommendations:
      rank=Rankings.loc[Rankings['asin']==item,'Rank']
      total+=rank.iloc[0]
      n+=1
  #Note: The highest rank of an item is 332 becaue items with equal no. of total ratings have similar ranks 
  return (total/n)


In [ ]:
%%time
print(Novelty(rankings,user_list))

In [ ]:
%%time
sim_df= pd.read_csv('/content/drive/My Drive/Data (1)/Similarity_Matrix_for_Diversity.csv')

In [ ]:
from itertools import combinations 

def Diversity(user_list,simsMatrix):
  print("Diversity: 1-S, where S is the average similarity score between every possible pair of recommendations")
  print("           for a given user. Higher means more diverse.")

  n=0
  total=0
  missed=0
  for u in tqdm(user_list):
    #the next 5 lines are to just generate the top n recommendations

    recs = rec.get_group(u).asin_Id.values
    recommendations=[id_to_asin[i] for i in recs]
    pairs = combinations(recommendations,2)
    for pair in pairs:

      item1= pair[0]
      item2= pair[1]

      if (item1 and item2 in sim_df.columns):

        similarity= simsMatrix.at[ int_to_asin[item1], item2]
        total+=similarity
        n+=1
      else:
        missed+=1


  print(n)
  S= total/n
  print('Total missed :')
  print(missed)

  return (1-S)
  


In [ ]:
%%time
print(Diversity(user_list,sim_df))